[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_income/00_economics_of_interest_rates.ipynb)

In [ ]:
## Installation

The following cell will install the `pyfian` package, which provides tools for financial analysis in Python. This step ensures that all necessary modules are available for the calculations and examples in this notebook.

In [ ]:
!git clone https://github.com/Python-Financial-Analyst/pyfian_dev.git
!pip install -e pyfian_dev/.

In [5]:
from pyfian.time_value.present_value import present_value_annuity
print(present_value_annuity(100, rate=0.1, periods=5))


from pyfian.time_value.present_value import present_value_growing_annuity
print(present_value_growing_annuity(100, growth=0.05, rate=0.1, periods=5))

from pyfian.time_value.mortgage_cash_flows import mortgage_cash_flows
mortgage_cash_flows(principal_balance=200000, annual_rate=0.04, term_months=10)

379.07867694084507
415.0591272329889


,Period,Payment,Interest,Principal,Remaining Balance
0,1,20368.5,666.67,19701.83,180298.17
1,2,20368.5,600.99,19767.50,160530.67
2,3,20368.5,535.10,19833.39,140697.27
3,4,20368.5,468.99,19899.51,120797.77
4,5,20368.5,402.66,19965.84,100831.93
5,6,20368.5,336.11,20032.39,80799.54
6,7,20368.5,269.33,20099.17,60700.37
7,8,20368.5,202.33,20166.16,40534.21
8,9,20368.5,135.11,20233.38,20300.83
9,10,20368.5,67.67,20300.83,0.00
